<a href="https://colab.research.google.com/github/Reennon/multigec-models/blob/main/notebooks/aya_expanse_8b/multigec/multigec_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

from google.colab import userdata

os.environ["GIT_TOKEN"] = userdata.get('git_token')

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://$GIT_TOKEN@github.com/Reennon/multigec-models.git

Cloning into 'multigec-models'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 115 (delta 34), reused 66 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 141.95 KiB | 1.80 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [4]:
%cd multigec-models

/content/multigec-models


In [5]:
!git pull

Already up to date.


In [6]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit -q
!pip install flash-attn --no-build-isolation -q
!pip install wtpsplit==2.1.1 -q
!pip install syntok==1.4.4 -q
!pip install omegaconf -q
!pip install wandb -q
!pip install --upgrade transformers trl -q
!pip install pandas numpy -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [8]:
import os

from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from src.utils.multigec import sentences, LANG_TO_CODE, LANG_CODE_TO_TOKEN
from langchain_core.prompts import PromptTemplate

from src.instruction_templates import multigec_prompts

import torch
import wandb

from transformers import BitsAndBytesConfig
from tqdm import tqdm
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

from peft import PeftModel, PeftModelForCausalLM

tqdm.pandas()

In [9]:
parameters = OmegaConf.load("./params/aya_expanse_8b.yaml")

In [10]:
track     = "minimal"
fine_tuned_model_name = f"aya-expanse-8b-multigec"

hf_key   = userdata.get("hf_key")
secret_wandb = userdata.get("wandb_key")

in_path  = f"/content/drive/MyDrive/multigec/datasets/multigec_{track}.csv"
out_path = f"/content/drive/MyDrive/multigec/preds/multigec_test_{track}.csv"

out_model_dir = f"/content/drive/MyDrive/multigec/models/{fine_tuned_model_name}"
QUANTIZE_4BIT = True
device   = "cuda:0"

In [11]:
login(hf_key)

In [12]:
!env TORCH_USE_CUDA_DSA=1 -q

env: ‘-q’: No such file or directory


In [15]:
base_model = "CohereForAI/aya-expanse-8b"
saved_checkpoint = out_model_dir + "/checkpoint-700"

quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(saved_checkpoint)
config = AutoConfig.from_pretrained(base_model)
base_model_instance = AutoModelForCausalLM.from_pretrained(
    base_model,
    config=config,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map=device,
    attn_implementation="flash_attention_2",
)
base_model_instance.resize_token_embeddings(len(tokenizer))
model = PeftModelForCausalLM.from_pretrained(
    base_model_instance,
    saved_checkpoint,
    torch_dtype=torch.bfloat16,
    device_map=device,
    ignore_mismatched_sizes=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:470: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([255040, 4096]) in the checkpoint and torch.Size([255029, 4096]) in the model instantiated
- base_model.model.model.embed_tokens.weight: found shape torch.Size([255040, 4096]) in the checkpoint and torch.Size([255029, 4096]) in the model instantiated.
  warnings.warn(msg)


In [16]:
multigec_df = pd.read_csv(in_path)
train_df = multigec_df.loc[multigec_df.loc[:, "split"] == "train"]
val_df = multigec_df.loc[multigec_df.loc[:, "split"] == "val"]
test_df = multigec_df.loc[multigec_df.loc[:, "split"] == "test"]

In [17]:
def formatting_prompts_func(example):
    language_code = LANG_TO_CODE[example["language"]]
    language_token = LANG_CODE_TO_TOKEN[language_code]

    user_input = example['feature']
    prompt_template = multigec_prompts[example["language"]].prompt_template
    instruction = prompt_template.format(original_text=user_input)

    text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>"

    return text

In [96]:
from torch.utils.data import DataLoader
from tqdm import tqdm

max_new_tokens = 1600
batch_size = 10
save_each = 200

# Assume test_df already exists and out_path is defined.
# Filter rows that need processing (target is empty/NaN).
to_process_df = test_df[test_df["target"].isna()].reset_index()  # preserve original index in "index" column

def collate_fn(examples):
    indices = [ex["index"] for ex in examples]
    texts = [formatting_prompts_func(example) for example in examples]
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    tokenized["indices"] = indices
    return tokenized


# Create DataLoader using only the rows that need processing.
dataloader = DataLoader(
    to_process_df.to_dict(orient="records"),
    batch_size=batch_size,
    collate_fn=collate_fn
)

processed_rows = 0
for batch in tqdm(dataloader):
    input_ids = batch["input_ids"].to(model.device)
    attention_mask = batch.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)
    prompt_padded_len = len(input_ids[0])

    gen_tokens = model.generate(
        input_ids,
        attention_mask=attention_mask,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=max_new_tokens,
        do_sample=True,
    )

    # Remove the prompt tokens from the generated tokens
    gen_tokens = [gt[prompt_padded_len:] for gt in gen_tokens]

    # Decode generated tokens to text corrections
    corrections = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    corrections = ["".join(c) for c in corrections]

    # Update the original DataFrame using the indices provided in the batch.
    for idx, corr in zip(batch["indices"], corrections):
        test_df.loc[idx, "target"] = corr
        processed_rows += 1

        if processed_rows % save_each == 0:
            test_df.to_csv(out_path, index=False)
            print(f"Saved progress after processing {processed_rows} rows.")

# Final save after processing all batches.
test_df.to_csv(out_path, index=False)
print("Final save complete!")


  0%|          | 0/18 [00:00<?, ?it/s]<ipython-input-96-fcb60d847550>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[idx, "target"] = corr
100%|██████████| 18/18 [55:53<00:00, 186.29s/it]

Final save complete!


,language,essay_id,split,feature,target


In [ ]:
test_df.to_csv(out_path)

In [52]:
test_df

Tak tady žiji ࿄ Z komínů stoupají dýmy spáleného uhlí nebo dříví, které bylo přivezeno samotnými obyvateli toho kterého domu, z nedalekého lesa. Jen z jednoho komínu se dým neubírá směrem vzhůru, ale zaplavuje dvorek domu sousedního, domu č. 17. Čas od času z tohoto domu vyjde jeden z jeho obyvatelů, přidusí se kouřem a vrátí se zpět do domu. To bývá v chladnějších měsících, kdy se musejí prostydlé staré domy vytápět. V teplých dnech se obyvatelé domu číslo 17 chodí dovnitř pouze zchladit, jinak tráví co nejvíce času venku – na dvorku či na zahradě. Podobně je to i v ostatních chalupách této malebné vesničky na blatech… ࿄ Vesnička se rozkládá v jakémsi „ďolíčku“ pod rozlehlým tmavým lesem, za který večer co večer zachází sluníčko v celé své červené kráse. Náves je zde skoro nenápadná, připomíná ji snad jen bílá kaplička, vedle níž se tyčí pomník věnovaný zdejším obyvatelům, kteří padli ve válkách. Jako v ostatních vesnicích zde nesmí chybět „rybník“, jenž je také součástí návsi. Do něj

In [57]:
test_df.reset_index()

,index,language,essay_id,split,feature,target
0,2,czech,tei-skript2012_10-cb1ajanjak_01_1,test,Osnova: I. ÚVOD – Co chci\nII. STAŤ: a) černé ...,NaN
1,8,czech,tei-skript2012_10-cb1akuband_02_1,test,Tak tady žiji ࿄ Z komínů stoupají dýmy spálené...,NaN
2,10,czech,tei-skript2012_10-cb1apexpet_01_1,test,: − pohled do zrcadla ࿄ −\n1) shrnutí obou obr...,NaN
3,14,czech,tei-skript2012_10-cb1ckaijan_02_1,test,Slohová práce školní\nJana Kaiserová\n17. květ...,NaN
4,17,czech,tei-skript2012_10-cb1cpolrob_02_1,test,Slohová práce školní\n17. května 2005\nTémata:...,NaN
...,...,...,...,...,...,...
2799,23984,ukrainian,1364,test,"Будь-яка активність, яку можна об'ярликувати ц...",NaN
2800,24219,ukrainian,1600,test,"""Як, - запитав я, - завадив вам?""\n\n- Своїм н...",NaN
2801,24266,ukrainian,1647,test,"Якби не ревіння води, якби не розкати грому, я...",NaN
2802,24332,ukrainian,1713,test,"Побачивши, як поводиться з вельми сумнівними в...",NaN


In [46]:
test_df[:1].target

,target
2,NaN


In [42]:
print(test_df[:1].feature.item())

Osnova: I. ÚVOD – Co chci
II. STAŤ: a) černé myšlenky
b) [1]„stav zóny“
c) rozhodující okamžik
III. ZÁVĚR – Co mi to dalo
ROZHODUJÍCÍ OKAMŽIK
Vyhrát!
Právě hraji nejdůležitější zápas mé dosavadní baseballové kariéry. Musíme vyhrát. Udělám pro to vše a mí spoluhráči také. Jsme silní.
Stav 20:16 pro naše soupeře. Podle vývoje hry se zdá, že jsme prohráli. Hrůzný pocit. Mám slzy v očích, ale nechci, aby to někdo viděl. Celá sezona pryč. Vše je pryč. Taková škoda. My to snad nikdy nedokážeme. Nikdy. Skoro brečím. V hlavě temno, chmurné myšlenky. Nic nevidím pozitivně jako vždy. Vidím před sebou jen celou tu dřinu. Naše snaha je zbytečná. Nemá to cenu. Trenéři jsou také „naměkko“. Topíme se ve vlastní beznaději. Teď už chci jenom usnout a na všechno zapomenout. Nechci zase zažít tu ostudu, že jsme něco nedokázali.
Najednou vidím všechno jakoby jinýma očima. Dostal jsem asi nějaký impulz. Je to poslední záchvěv nebo nová síla? Je to zvláštní. Necítím únavu. Ta síla mne tlačí dál. A nejsem sá

In [43]:
print(all_corrections[0])

Osnova: I. ÚVOD – Co chci
II. STAŤ: a) černé myšlenky
b) [1]„stav zóny“
c) rozhodující okamžik
III. ZÁVĚR – Co mi to dalo
ROZHODUJÍCÍ OKAMŽIK
Vyhrát!
Právě hraju nejdůležitější zápas mé dosavadní baseballové kariéry. Musíme vyhrát. Udělám pro to vše a mí spoluhráči také. Jsme silní.
Stav 20:16 pro naše soupeře. Podle vývoje hry se zdá, že jsme prohráli. Hrůzný pocit. Mám slzy v očích, ale nechci, aby to někdo viděl. Celá sezona pryč. Vše je pryč. Taková škoda. My to snad nikdy nedokážeme. Nikdy. Skoro brečím. V hlavě temno, chmurné myšlenky. Nic nevidím pozitivně jako vždy. Vidím před sebou jen celou tu dřinu. Naše snaha je zbytečná. Nemá to cenu. Trenéři jsou také „naměkko“. Topíme se ve vlastní beznaději. Teď už chci jenom usnout a na všechno zapomenout. Nechci zase zažít tu ostudu, že jsme něco nedokázali.
Najednou vidím všechno jakoby jinýma očima. Dostal jsem asi nějaký impulz. Je to poslední záchvěv nebo nová síla? Je to zvláštní. Necítím únavu. Ta síla mne tlačí dál. A nejsem sá

In [31]:
"".join(all_corrections)

'<|START_OF_TURN_TOKEN|><|USER_TOKEN|><|LANG_CS|>Opravte všechny pravopisné, interpunkční, stylistické, gramatické, lexikální a syntaktické chyby v následujícím textu. \nPokud v něm nejsou žádné chyby, text jednoduše zopakujte. \nV odpovědi uveďte pouze opravený text bez dalších vysvětlivek a komentářů.\n\nOsnova: I. ÚVOD – Co chci\nII. STAŤ: a) černé myšlenky\nb) [1]„stav zóny“\nc) rozhodující okamžik\nIII. ZÁVĚR – Co mi to dalo\nROZHODUJÍCÍ OKAMŽIK\nVyhrát!\nPrávě hraji nejdůležitější zápas mé dosavadní baseballové kariéry. Musíme vyhrát. Udělám pro to vše a mí spoluhráči také. Jsme silní.\nStav 20:16 pro naše soupeře. Podle vývoje hry se zdá, že jsme prohráli. Hrůzný pocit. Mám slzy v očích, ale nechci, aby to někdo viděl. Celá sezona pryč. Vše je pryč. Taková škoda. My to snad nikdy nedokážeme. Nikdy. Skoro brečím. V hlavě temno, chmurné myšlenky. Nic nevidím pozitivně jako vždy. Vidím před sebou jen celou tu dřinu. Naše snaha je zbytečná. Nemá to cenu. Trenéři jsou také „naměkko“.

In [25]:
test_df[:10]

,language,essay_id,split,feature,target
2,czech,tei-skript2012_10-cb1ajanjak_01_1,test,Osnova: I. ÚVOD – Co chci\nII. STAŤ: a) černé ...,NaN
8,czech,tei-skript2012_10-cb1akuband_02_1,test,Tak tady žiji ࿄ Z komínů stoupají dýmy spálené...,NaN
10,czech,tei-skript2012_10-cb1apexpet_01_1,test,: − pohled do zrcadla ࿄ −\n1) shrnutí obou obr...,NaN
14,czech,tei-skript2012_10-cb1ckaijan_02_1,test,Slohová práce školní\nJana Kaiserová\n17. květ...,NaN
17,czech,tei-skript2012_10-cb1cpolrob_02_1,test,Slohová práce školní\n17. května 2005\nTémata:...,NaN
18,czech,tei-skript2012_10-cb1cratsta_02_1,test,Slohová práce školní\nStanislav Rataj\n17. kvě...,NaN
19,czech,tei-skript2012_10-cb1csebren_02_1,test,Slohová práce školní\nRené Šebl\n17. května 20...,NaN
23,czech,tei-skript2012_10-cb1czikver_02_1,test,Slohová práce školní\n1. C\n17. května 2005\nT...,NaN
26,czech,tei-skript2012_10-grp1bdrbpav_1,test,27. listopadu 2008\nPavel Drbohlav P1b\n1. slo...,NaN
31,czech,tei-skript2012_10-grp1bmaykat_1,test,27. listopadu 2008\nNováková Eva\nP1b\n1. píse...,NaN


In [ ]:
test_df.to_csv(out_path)

In [ ]:
def make_correction(
    model,
    input: pd.Series,
    tokenizer,
    parameters: dict,
    seq_lenght: int
):
    input = formatting_prompts_func(input)
    input_ids = tokenizer(
        [input],
        padding=True,
        return_tensors="pt",
    ).input_ids
    input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])

    # Generate corrections
    gen_tokens = model.generate(
        input_ids,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=seq_lenght,
        do_sample=True,
    )
    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[str] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    # Join divided texts if any
    correction = "".join(outputs)

    return correction

test_df[:5]["target"] = test_df[:5].progress_apply(lambda x: make_correction(
    model=model,
    input=x,
    tokenizer=tokenizer,
    parameters=parameters,
    seq_lenght=1600,
), axis=1)
test_df

100%|██████████| 5/5 [07:38<00:00, 91.65s/it]
<ipython-input-102-0620bec5f089>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[:5]["target"] = test_df[:5].progress_apply(lambda x: make_correction(


,language,essay_id,split,feature,target
2,czech,tei-skript2012_10-cb1ajanjak_01_1,test,Osnova: I. ÚVOD – Co chci\nII. STAŤ: a) černé ...,ebezpečí\nVyhrát!\nPrávě hraji nejdůležitější ...
8,czech,tei-skript2012_10-cb1akuband_02_1,test,Tak tady žiji ࿄ Z komínů stoupají dýmy spálené...,NaN
10,czech,tei-skript2012_10-cb1apexpet_01_1,test,: − pohled do zrcadla ࿄ −\n1) shrnutí obou obr...,NaN
14,czech,tei-skript2012_10-cb1ckaijan_02_1,test,Slohová práce školní\nJana Kaiserová\n17. květ...,NaN
17,czech,tei-skript2012_10-cb1cpolrob_02_1,test,Slohová práce školní\n17. května 2005\nTémata:...,NaN
...,...,...,...,...,...
23984,ukrainian,1364,test,"Будь-яка активність, яку можна об'ярликувати ц...",NaN
24219,ukrainian,1600,test,"""Як, - запитав я, - завадив вам?""\n\n- Своїм н...",NaN
24266,ukrainian,1647,test,"Якби не ревіння води, якби не розкати грому, я...",NaN
24332,ukrainian,1713,test,"Побачивши, як поводиться з вельми сумнівними в...",NaN


In [ ]:
test_df[:1].feature.item()

Osnova: I. ÚVOD – Co chci
II. STAŤ: a) černé myšlenky
b) [1]„stav zóny“
c) rozhodující okamžik
III. ZÁVĚR – Co mi to dalo
ROZHODUJÍCÍ OKAMŽIK
Vyhrát!
Právě hraji nejdůležitější zápas mé dosavadní baseballové kariéry. Musíme vyhrát. Udělám pro to vše a mí spoluhráči také. Jsme silní.
Stav 20:16 pro naše soupeře. Podle vývoje hry se zdá, že jsme prohráli. Hrůzný pocit. Mám slzy v očích, ale nechci, aby to někdo viděl. Celá sezona pryč. Vše je pryč. Taková škoda. My to snad nikdy nedokážeme. Nikdy. Skoro brečím. V hlavě temno, chmurné myšlenky. Nic nevidím pozitivně jako vždy. Vidím před sebou jen celou tu dřinu. Naše snaha je zbytečná. Nemá to cenu. Trenéři jsou také „naměkko“. Topíme se ve vlastní beznaději. Teď už chci jenom usnout a na všechno zapomenout. Nechci zase zažít tu ostudu, že jsme něco nedokázali.
Najednou vidím všechno jakoby jinýma očima. Dostal jsem asi nějaký impulz. Je to poslední záchvěv nebo nová síla? Je to zvláštní. Necítím únavu. Ta síla mne tlačí dál. A nejsem sá

In [ ]:
print(test_df[:1].target.item())

ebezpečí
Vyhrát!
Právě hraji nejdůležitější zápas mé dosavadní baseballové kariéry. Musíme vyhrát. Udělám pro to vše a mí spoluhráči také. Jsme silní.
Stav 20:16 pro naše soupeře. Podle vývoje hry se zdá, že jsme prohráli. Hrůzný pocit. Mám slzy v očích, ale nechci, aby to někdo viděl. Celá sezona pryč. Vše je pryč. Taková škoda. My to snad nikdy nedokážeme. Nikdy. Skoro brečím. V hlavě temno, chmurné myšlenky. Nic nevidím pozitivně jako vždy. Vidím před sebou jen celou tu dřinu. Naše snaha je zbytečná. Nemá to cenu. Trenéři jsou také „naměkko“. Topíme se ve vlastní beznaději. Teď už chci jenom usnout a na všechno zapomenout. Nechci zase zažít tu ostudu, že jsme něco nedokázali.
Najednou vidím všechno jakoby jinýma očima. Dostal jsem asi nějaký impulz. Je to poslední záchvěv nebo nová síla? Je to zvláštní. Necítím únavu. Ta síla mne tlačí dál. A nejsem sám! Najednou všichni jakoby ožívají. Týmu se zmocňuje tzv. Naganský duch. Nevymýšlím si. Nefantazíruji. Je to tak. Je to realita. Vidí

In [ ]:
raise Exception(e)

In [ ]:
def formatting_prompts_func(example):
    language_code = LANG_TO_CODE[example["language"]]
    language_token = LANG_CODE_TO_TOKEN[language_code]

    user_input = example['feature']
    prompt_template = multigec_prompts[example["language"]].prompt_template
    instruction = prompt_template.format(original_text=user_input)
    target = example['target']

    text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"

    return text

In [ ]:
num_added_toks = tokenizer.add_tokens(
    [v for v in LANG_CODE_TO_TOKEN.values()],
    special_tokens=True
)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
training_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

seq_length = 1300

cld_train_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=training_dataset,
    #dataset_text_field='feature',
    seq_length=seq_length,
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)
cld_val_dataset = ConstantLengthDataset(
    tokenizer=tokenizer,
    dataset=val_dataset,
    #dataset_text_field='feature',
    seq_length=int(seq_length/2),
    eos_token_id=tokenizer.eos_token_id,
    shuffle=True,
    append_concat_token=True,
    add_special_tokens=True,
    formatting_func=formatting_prompts_func,
)

In [ ]:
# parameters.training["gradient_accumulation_steps"] = 8
# parameters.training["per_device_train_batch_size"] = 5
# parameters.training["per_device_eval_batch_size"] = 2
# # Try these
# parameters.training["lr_scheduler_type"] = "cosine"
# parameters.training["max_grad_norm"] = 1.0
# parameters.training["output_dir"] = "results"
# parameters.training["num_train_epochs"] = 12
# parameters.training["eval_strategy"] = "steps"
# parameters.training["save_strategy"] = "steps"        # Save model after every step
# parameters.training["metric_for_best_model"] = "eval_loss"
# parameters.training["greater_is_better"] = False
# parameters.training["group_by_length"] = False # we already use CLD
# parameters.training["save_total_limit"] = 3           # Keep only the 3 most recent checkpoints
# parameters.training["load_best_model_at_end"] = True # EarlyStoppingCallback requires load_best_model_at_end = True
# parameters.training["warmup_steps"] = 50
# parameters.training["learning_rate"] = 3e-5
# parameters.training["save_steps"] = 25
# parameters.training["weight_decay"] = 0.0
# early_stopping_patience = 75

In [ ]:
cld_train_dataset

In [ ]:
dict(parameters.training)

In [ ]:
# def formatting_prompts_func_old(example):
#     output_texts = []
#     for i in range(len(example['inputs'])):
#         language_code = LANG_TO_CODE[example["language"]]
#         language_token = LANG_CODE_TO_TOKEN[language_code]

#         user_input = example['inputs'][i]
#         prompt_template = multigec_prompts[example["language"]]
#         instruction = prompt_template.format(original_text=user_input)
#         target = example['targets'][i]


#         text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"
#         output_texts.append(text)
#     return output_texts

# def formatting_prompts_func(example):
#     language_code = LANG_TO_CODE[example["language"]]
#     language_token = LANG_CODE_TO_TOKEN[language_code]

#     user_input = example['feature']
#     prompt_template = multigec_prompts[example["language"]]
#     instruction = prompt_template.format(original_text=user_input)
#     target = example['target']

#     text = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{language_token}{instruction}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{target}"

#     return text

In [ ]:
run = wandb.init(
    project=wandb_project_name,
    job_type="training",
    anonymous="allow"
)

wandb.config.update(dict(parameters.training))

peft_config = LoraConfig(
    r=parameters.lora.r,
    lora_alpha=parameters.lora.lora_alpha,
    target_modules=list(parameters.lora.target_modules),
    bias=parameters.lora.bias,
    task_type=parameters.lora.task_type
)
training_arguments = SFTConfig(
    **parameters.training,
    packing=True,
    max_seq_length=seq_length,
    output_dir=out_model_dir,
)
trainer = SFTTrainer(
    model=model,
    train_dataset=cld_train_dataset,
    eval_dataset=cld_val_dataset,
    peft_config=peft_config,
    args=training_arguments,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=parameters.early_stopping.early_stopping_patience)],
    #formatting_func=formatting_prompts_func,
)

with torch.backends.cuda.sdp_kernel(
    enable_flash=True,
    enable_math=False,
    enable_mem_efficient=False
):
    trainer.unload() # Should remove peft warnings
    trainer.train(resume_from_checkpoint=True)

In [ ]:
import time

run_name = wandb.run.name
model_out_path = f"/gdrive/MyDrive/models/multigec/{run_name}"
os.makedirs(model_out_path, exist_ok=True)
trainer.model.save_pretrained(model_out_path)
wandb.save(model_out_path + "/*")
model.config.use_cache = True
model.eval()
time.sleep(30)

In [ ]:
test_df = multigec_df.loc[multigec_df.loc[:, "split"] == "test"]

def make_correction(
    model,
    input: pd.Series,
    tokenizer,
    parameters: dict,
    seq_lenght: int
):
    language: str = input["language"]
    language_code: str = LANG_TO_CODE[language]
    language_token: str = LANG_CODE_TO_TOKEN[language_code]
    prompt_template: PromptTemplate = multigec_prompts[language]

    # Prepare inputs
    inputs: list[str] = sentences(input)
    inputs = [language_token + prompt_template.format(original_text=input) for input in inputs]
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])

    # Generate corrections
    gen_tokens = model.generate(
        input_ids,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=seq_lenght,
        do_sample=True,
    )
    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[str] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    # Join divided texts if any
    correction = "".join(outputs)

    return correction

test_df.progress_apply(lambda x: make_correction(
    model=model,
    input=x,
    tokenizer=tokenizer,
    parameters=parameters,
    seq_lenght=seq_length,
))
test_df

In [ ]:
test_df.to_csv(out_path)

In [ ]:
wandb.finish()

In [ ]:
raise Exception e

In [95]:
!nvidia-smi

Sun Mar 30 19:32:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             49W /  400W |   19715MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [94]:
torch.cuda.empty_cache()

In [ ]:
!pip install pynvml -q

In [ ]:
from google.colab import runtime
runtime.unassign()

In [93]:
import gc

gc.collect()

0

In [78]:
import time
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

def wait_until_enough_gpu_memory(min_memory_available, max_retries=10, sleep_time=5):
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(torch.cuda.current_device())

    for _ in range(max_retries):
        clear_gpu_memory()
        info = nvmlDeviceGetMemoryInfo(handle)
        if info.free >= min_memory_available:
            break
        print(f"Waiting for {min_memory_available} bytes of free GPU memory. Retrying in {sleep_time} seconds...")
        time.sleep(sleep_time)
    else:
        raise RuntimeError(f"Failed to acquire {min_memory_available} bytes of free GPU memory after {max_retries} retries.")

# Usage example
min_memory_available = 38 * 1024 * 1024 * 1024  # 30GB
clear_gpu_memory()
wait_until_enough_gpu_memory(min_memory_available)

NameError: name 'clear_gpu_memory' is not defined

In [ ]:
del model

In [ ]:
del trainer